In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

from source_code.SqlCo import Sqldd

Autosave disabled


In [2]:
## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
282,AFG,Asia,Afghanistan,2020-10-08,39616.0,68.0,49.714,1470.0,1.0,1.714,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
496,ALB,Europe,Albania,2020-10-08,14730.0,162.0,154.429,407.0,4.0,2.857,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
779,DZA,Africa,Algeria,2020-10-08,52520.0,121.0,141.429,1779.0,6.0,6.429,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [3]:
## dump the file for using it in other nb
# c.to_csv(f"data\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## Load data from days -1 to evaluate the performances of model
yesterday_date = date.today() - timedelta(days=1)
yesterday_date = str(yesterday_date)

tip = Sqldd()
cnx, cursor = tip.get_bdd_co()
data_load = pd.read_sql(f"SELECT * FROM cov_aipred WHERE date='{yesterday_date}';", con=cnx) 

In [5]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    res_tempo= res_tempo.reset_index()
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.at[0,"date"] = v0
    follow_df.at[0,"country"] = country
    follow_df.at[0,"total_cases_predict"] = rez1
    follow_df.at[0,"total_cases_real"] = v1
    follow_df.at[0,"total_deaths_predict"] = rez2
    follow_df.at[0,"total_deaths_real"] = v2
    follow_df.at[0,"error_abs_cases"] = errorCase1
    follow_df.at[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [6]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [7]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-08,France,642397.0,653509.0,32434.0,32445.0,-11112.0,-11.0
0,2020-10-08,China,90687.0,90689.0,4743.0,4739.0,-2.0,4.0
0,2020-10-08,Italy,332143.0,333940.0,36060.0,36061.0,-1797.0,-1.0
0,2020-10-07,Spain,838909.0,835901.0,32619.0,32562.0,3008.0,57.0
0,2020-10-08,United States,7577800.0,7549794.0,211905.0,211801.0,28006.0,104.0
0,2020-10-08,World,36409600.0,36194764.0,1057130.0,1053961.0,214836.0,3169.0
0,2020-10-08,United Kingdom,540873.0,544275.0,42486.0,42515.0,-3402.0,-29.0
0,2020-10-08,Germany,308285.0,310144.0,9576.0,9578.0,-1859.0,-2.0
0,2020-10-08,Iran,482634.0,483844.0,27614.0,27658.0,-1210.0,-44.0
0,2020-10-08,Turkey,328788.0,329138.0,8601.0,8609.0,-350.0,-8.0


In [8]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
# rapport.to_csv(f"data\\Rapport\\rap{date.today()}.csv")

In [9]:
## insert yesterday reporting to bdd
# try:
#     data_job = data_up_pip(rapport) 
#     data_job.up_rapp()
#     data_job.clean_leave()
# except:
#     print("BDD job Failed -- Normal if you not have acces to main code")